###  Imports

In [1]:
import torch

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset
from model import create_model
from utils import add_detections, get_transforms

import config

torch.manual_seed(1)

In [2]:
print(fo.list_datasets())

[]


In [3]:
if len(fo.list_datasets()) > 0:
    dataset = fo.load_dataset("coco-2017-validation")
    dataset.delete()

In [4]:
#Load in the dataset from the FiftyOne model Zoo
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

#needed to calculate image height and width
fo_dataset.compute_metadata()

session = fo.launch_app(fo_dataset)

subset = False

train_transforms, test_transforms = get_transforms()

if subset:
    # to filter certain items from the dataset we can
    item_list = ["car", "dog", "bus", 'fork', 'tie', 'person', 'airplane']
    item_list = ["man", "boy", "skis"]
    #item_list = ['airplane', 'cat']
    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(item_list))

    #session.view = item_view

    # split the dataset in train and test set
    train_view = item_view.take((len(item_view) * config.TRAIN_TEST_SPLIT), seed=51)
    test_view = item_view.exclude([s.id for s in train_view])

else:
    train_view = fo_dataset.take(len(fo_dataset) * config.TRAIN_TEST_SPLIT)
    test_view = fo_dataset.exclude([s.id for s in train_view])
    item_list = fo_dataset.distinct("ground_truth.detections.label")

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=item_list)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms,
        classes=item_list)

# #this is needed for later use, but not for creating the dataset
# if item_list[0] != 'background':
#      item_list.insert(0,'background')

Found annotations at 'C:\Users\blain\fiftyone\coco-2017\raw\instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [15.7s elapsed, 0s remaining, 332.3 samples/s]      
Dataset 'coco-2017-validation' created


Traning on 4000 samples
Testing on 1000 samples


# Check Faster RCNN performance

In [5]:
# MODEL_TYPE = 'CLIP-Backbone-FRCNN'
# CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_28.pth'
#
# if item_list[0] != 'background':
#      item_list.insert(0,'background')
#
# frcnn_model = create_model(MODEL_TYPE, classes=item_list)
# checkpoint = torch.load(CHECKPOINT_NAME)
# frcnn_model = create_model(MODEL_TYPE, classes=item_list)
#
# frcnn_model.load_state_dict(checkpoint)
# frcnn_model.eval()
#
# add_detections(frcnn_model, evaluation_dataset, fo_dataset, field_name="frcnn_predictions")
#
# results = fo.evaluate_detections(
#     test_view,
#     "frcnn_predictions",
#     classes=item_list,
#     eval_key="eval",
#     compute_mAP=True
# )
# session.view = item_view
# print(f'mAP: {results.mAP()}')
# results.print_report()

# Check CLIP FRCNN performance

In [ ]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-RPN'
WEIGHTS_NAME = 'rpn'




# tokenize item list for CLIP
import clip
_, preprocess = clip.load("RN50", device=config.DEVICE)

if item_list[0] != '':
     item_list.insert(0,' ')

text_tokens = clip.tokenize(["This is " + desc for desc in item_list]).cuda()

CHECKPOINT_NAME = f'{MODEL_TYPE}_{WEIGHTS_NAME}.pth'
checkpoint = torch.load(CHECKPOINT_NAME)
clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)

clip_frcnn_model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
print(f'loaded checkpoint at epoch {epoch}')

clip_frcnn_model.eval()

In [7]:
# find performance without clustering
add_detections(clip_frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_RPN_predictions_no_clustering", PRED_CLUSTERING=False)

results = fo.evaluate_detections(
    test_view,
    "clip_RPN_predictions_no_clustering",
    classes=item_list,
    eval_key="clip_eval_no_clustering",
    compute_mAP=True
)

print(f'mAP: {results.mAP()}')
results.print_report()

session.view = test_view

Using device cuda
 100% |███████████████| 1000/1000 [2.0m elapsed, 0s remaining, 8.6 samples/s]      
Evaluating detections...
 100% |███████████████| 1000/1000 [35.2s elapsed, 0s remaining, 27.9 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1000/1000 [1.1m elapsed, 0s remaining, 18.2 samples/s]      
mAP: 0.0076217544159791085
                precision    recall  f1-score   support

                     0.00      0.00      0.00         0
      airplane       0.01      0.89      0.01        27
         apple       0.04      0.17      0.07        60
      backpack       0.01      0.07      0.02        86
        banana       0.17      0.36      0.23        88
  baseball bat       0.01      0.04      0.01        24
baseball glove       0.00      0.00      0.00        24
          bear       0.04      0.20      0.07        10
           bed       0.02      0.63      0.05        49
         bench       0.00      0.11      0.01        72
       bicycle       0.02      0.3

In [6]:
# test out the trained CLIP-FRCNN with pred_clustering

add_detections(clip_frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_RPN_predictions_with_clustering", PRED_CLUSTERING=True)

results = fo.evaluate_detections(
    test_view,
    "clip_RPN_predictions_with_clustering",
    classes=item_list,
    eval_key="clip_eval_with_clustering",
    compute_mAP=True
)

print(f'mAP: {results.mAP()}')
results.print_report()

session.view = test_view

loaded checkpoint at epoch 23
Using device cuda
   0% |/--------------|    1/1000 [314.5ms elapsed, 5.2m remaining, 3.2 samples/s] 

C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |███████████████| 1000/1000 [2.1m elapsed, 0s remaining, 8.4 samples/s]      
Evaluating detections...
 100% |███████████████| 1000/1000 [6.7s elapsed, 0s remaining, 149.3 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1000/1000 [10.2s elapsed, 0s remaining, 97.9 samples/s]      
mAP: 0.010486062785356395
                precision    recall  f1-score   support

                     0.00      0.00      0.00         0
      airplane       0.03      0.37      0.05        27
         apple       0.06      0.06      0.06        54
      backpack       0.00      0.00      0.00        86
        banana       0.20      0.08      0.12        61
  baseball bat       0.05      0.04      0.04        24
baseball glove       0.00      0.00      0.00        24
          bear       0.12      0.10      0.11        10
           bed       0.14      0.33      0.20        49
         bench       0.01      0.03      0.01        72
       bicycle       0.09      0.09      0.09       